In [1]:
import numpy as np
import pandas as pd
import _pickle as pickle
from collections import defaultdict
import re
import ast

from bs4 import BeautifulSoup

import sys
import os
import json
import operator

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

from keras.optimizers import SGD

from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Dropout, Activation, Input, Flatten, Concatenate, GlobalAveragePooling1D, LSTM, GRU, Bidirectional, dot, multiply, Lambda, TimeDistributed, Masking

from keras.models import Model, Sequential
from keras.regularizers import l2,l1,l1_l2
from keras.callbacks import Callback,EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight

from keras.callbacks import Callback

from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, classification_report,accuracy_score, auc, roc_curve, roc_auc_score, average_precision_score


Using TensorFlow backend.


In [2]:
def load_data(data_name):
    f = open(data_name,'rb')
    data = pickle.load(f)
    return data

In [3]:
data = load_data('data_20200212_2012')

In [4]:
text_para = []
labels = []
for d in data:
    text_para.append(d[0])
    labels.append(d[1])

In [5]:
def generate_doc(x):
    return ' '.join(x)

In [6]:
text_doc = []
for d in text_para:
    text_doc.append(generate_doc(d))

In [ ]:
# labels = load_data('label_20200206_2012')

In [ ]:
# print('labels',len(labels))

In [ ]:
#para_text = load_data('para_20200206_2012')

In [ ]:
#print('para',len(para_text))

In [ ]:
#doc_text = load_data('doc_20200206_2012')

In [ ]:
#print('doc',len(doc_text))

In [7]:
MAX_PARA_LENGTH = 250
MAX_PARAS = 200
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
#l1_reg = l1(1e-5)
l12_reg = l1_l2(l1 = 1e-6,l2 = 1e-6)
BATCH_SIZE = 24
#metrics_auc = {}
metrics_prcs = {}
num_folder = 5
#date = '20200208'

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text_doc)

In [9]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 201564 unique tokens.


In [14]:
word_index_50000 = { k:v for (k,v) in word_index.items() if v<=50000 }

In [11]:
with open('/home/jujun/fraudprediction_10k/HAN/input_data/word_index_50000_20200212','wb') as fp:
    pickle.dump(word_index_50000,fp)
fp.close()

In [12]:
# with open('/home/jujun/fraudprediction_10k/HAN/input_data/word_index_50000_20200207','rb') as fp:
#     word_index_5000 = pickle.load(fp)

In [15]:
type(word_index_50000)

dict

In [16]:
def get_word(value):
    return {k:v for k,v in word_index_50000.items() if v == value}

In [17]:
data = np.zeros((len(labels), MAX_PARAS, MAX_PARA_LENGTH), dtype='int32') 

In [18]:
data.shape

(56288, 200, 250)

In [19]:
for i, doc in enumerate(text_para):
    #structure of a doc: [p1,p2,p3,p4]
    for j, para in enumerate(doc):
        
        if j < MAX_PARAS:
            wordTokens = text_to_word_sequence(para)
            
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_PARA_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [ ]:
# data = np.load('data_20200206.npy')

In [20]:
doc_w2v = [x.split(' ') for x in text_doc]
from gensim.models import Word2Vec

In [42]:
#Get my W2V model
#Parameter: window=5,size=EMBEDDING_DIM, sg=1, workers=os.cpu_count(),min_count=5
w2v_model = Word2Vec(doc_w2v, window=5, workers=os.cpu_count(), min_count=5, size=EMBEDDING_DIM)

In [43]:
embedding_matrix = np.random.random(((MAX_NB_WORDS+ 1) , EMBEDDING_DIM))
ignored=[]
cnt=0
for word, i in word_index.items():
    if word in w2v_model.wv.vocab and tokenizer.word_index[word] < MAX_NB_WORDS:
        embedding_vector = w2v_model[word]
        cnt+=1
        if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    else: ignored.append(word)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [44]:
len(w2v_model.wv.vocab)

120152

In [45]:
cnt

49334

In [46]:
np.save('embedding_matrix_20200212', embedding_matrix)

In [47]:
w2v_model.save('w2v_model_202002012.model')

In [48]:
# embedding_matrix = np.load('embedding_matrix_20200206.npy')

In [49]:
def performance_measure(pred_yp, y):
    '''
    Given lists of predicted y probability and x, y, return a dataframe of AR, AUC, Brier, Decile Table
    '''
    
    tenc_dat = pd.DataFrame({'y_true':y,'probability':pred_yp.flatten()})
    tenc_dat.sort_values('probability',axis = 0,ascending=False, inplace = True)
    tenc_dat.index = range(0,len(tenc_dat))
    y = tenc_dat['y_true']
    point = float(len(tenc_dat))/10
    point = int(round(point))
    tenc = []
    for i in range(0,10):
        tenc.append(y[(i*point):((i+1)*point)])
    tenc[9]=tenc[9].append(y[10*point:])
    total = sum(y)
    num_of_bkr = []
    for j in range(0,10):
        num_of_bkr.append(sum(tenc[j]))
    tencile_bkr = np.array(num_of_bkr)
    rate = tencile_bkr.astype(float)/total

    return rate

In [50]:
class AUCEvaluation(Callback):
    """ Show AUC after interval number of epoches """
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            logs['auc'] = score
            tencile=performance_measure(y_pred, self.y_val)
            logs['tencile'] = tencile
            print(" epoch:{:d} AUC: {:.4f}".format(epoch, score))

In [51]:
class PrecisionEvaluation(Callback):
    """ Show average Precision after interval number of epoches """
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = average_precision_score(self.y_val, y_pred)
            logs['Avg_Prec'] = score
            #tencile=performance_measure(y_pred, self.y_val)
            #logs['tencile'] = tencile
            #print('prec_socre',score)
            print(" epoch:{:d} Avg_Prec: {:.4f}".format(epoch, score))

In [52]:
class AUCEvaluation(Callback):
    """ Show AUC after interval number of epoches """
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            logs['auc'] = score
            tencile=performance_measure(y_pred, self.y_val)
            logs['tencile'] = tencile
            print(" epoch:{:d} AUC: {:.4f}".format(epoch, score))

In [53]:
class PrecisionEvaluation(Callback):
    """ Show average Precision after interval number of epoches """
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = average_precision_score(self.y_val, y_pred)
            logs['Avg_Prec'] = score
            #tencile=performance_measure(y_pred, self.y_val)
            #logs['tencile'] = tencile
            #print('prec_socre',score)
            print(" epoch:{:d} Avg_Prec: {:.4f}".format(epoch, score))

In [54]:
class AttLayer(Layer):
    
    def __init__(self, regularizer=None,context_dim=100, name="attention",**kwargs):
        self.regularizer = regularizer
        self.context_dim=context_dim
        self.supports_masking = True
        self.name=name
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3        
        self.W = self.add_weight(name='W', shape=(input_shape[-1], self.context_dim), initializer='normal', trainable=True, 
                                 regularizer=self.regularizer)
        self.b = self.add_weight(name='b', shape=(self.context_dim,), initializer='normal', trainable=True, 
                                 regularizer=self.regularizer)
        self.u = self.add_weight(name='u', shape=(self.context_dim,), initializer='normal', trainable=True, 
                                 regularizer=self.regularizer)        
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W) + self.b)
        eij = K.squeeze(K.dot(eij, K.expand_dims(self.u, axis=1)), axis=-1)
        #eij = K.dot(K.tanh(K.dot(x, self.W) + self.b), self.u)
        ai = K.exp(eij)
        #print("ai size", K.int_shape(ai))
        #print("mask size", K.int_shape(mask))
        
        if mask is not None:
            ai*=K.cast(mask, K.floatx())
            
        ai /=K.cast(K.sum(ai, axis=1, keepdims=True)+K.epsilon(), K.floatx())
        #weights=K.expand_dims(weights, axis=2) # (None, 1000,1)
        #alphas = ai / K.sum(ai, axis=1).dimshuffle(0, 'x')
        
        #weighted_input = x * K.expand_dims(ai)
        #return K.sum(weighted_input, axis=1)
        return ai
        

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])
    
    def get_config(self):
        config = {}
        base_config = super(AttLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_mask(self, inputs, mask):
        return None    

In [55]:
class WeightedSum(Layer):
    def __init__(self, name="weighted_sum",  **kwargs):
        self.supports_masking = True
        self.name=name
        super(WeightedSum, self).__init__(**kwargs)
        
    def build(self, input_shape):
        #assert isinstance(input_shape, list)
        #print("input shape", input_shape)
        super(WeightedSum, self).build(input_shape)  # Be sure to call this at the end


    def call(self, input_tensor, mask=None):
        
        x = input_tensor[0]
        #print("input",K.int_shape(x))
        a = input_tensor[1]
        print("weights", K.int_shape(a))

        a = K.expand_dims(a)
        weighted_input = K.sum(x * a, axis=1)
        #print("weighted sum",K.int_shape(weighted_input))
        
        return weighted_input

    def compute_output_shape(self, input_shape):
        
        a, b = input_shape
        
        return (a[0], a[-1])

    def compute_mask(self, x, mask=None):
        return None

In [56]:
def basicModel(embedding_matrix,MAX_NB_WORDS,MAX_PARA_LENGTH,):
    embedding_layer = Embedding((MAX_NB_WORDS+ 1),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            mask_zero=True,
                            input_length=MAX_PARA_LENGTH,
                            trainable=True)    
    
    
    para_input = Input(shape=(MAX_PARA_LENGTH, ), dtype='int32')
    embedded_sequences = embedding_layer(para_input)
    norm_sequence = BatchNormalization()(embedded_sequences)
    drop_out = Dropout(0.1)(norm_sequence)     
    l_att = AttLayer(regularizer = l12_reg)(drop_out)
    weighted_sum = WeightedSum()([norm_sequence,l_att])
    paraEncoder =Model(para_input,weighted_sum)
    paraEncoder.summary()
    
    doc_input = Input(shape=(MAX_PARAS, MAX_PARA_LENGTH), dtype='int32')
    doc_encoder = TimeDistributed(paraEncoder)(doc_input)
    mask_doc = Masking(mask_value=0.0)(doc_encoder)
    norm_doc = BatchNormalization()(mask_doc)
    l_lstm_para = GRU(100, return_sequences=True,implementation=2,kernel_regularizer=l12_reg)(norm_doc)
    norm_doc_1 = BatchNormalization()(l_lstm_para)
    l_att_para = AttLayer(regularizer=l12_reg)(norm_doc_1)
    weighted_sum_doc = WeightedSum()([norm_doc_1, l_att_para])
    batch_norm_doc = BatchNormalization()(weighted_sum_doc)
    drop_out_doc = Dropout(0.1)(batch_norm_doc)

    preds = Dense(1, activation='sigmoid',kernel_regularizer=l12_reg)(drop_out_doc) 

    model = Model(doc_input, preds)
    model.summary()
    
    return model

In [57]:
def trainModel(x_train,y_train, Best_Model_Filepath,model,x_val, y_val):
    class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
    
    auc_eval = AUCEvaluation(validation_data=(x_val, y_val), interval=1)
    precision_eval = PrecisionEvaluation(validation_data=(x_val, y_val), interval=1)
    
    earlyStopping = EarlyStopping(monitor='Avg_Prec',patience = 5, verbose =2, mode ='max')
    checkpoint = ModelCheckpoint(Best_Model_Filepath, monitor='Avg_Prec', verbose=2, save_best_only=True, mode ='max')
    print("training start...")
    training = model.fit(x_train,y_train,epochs=50,batch_size=BATCH_SIZE, \
                         callbacks=[auc_eval, precision_eval, earlyStopping, checkpoint],class_weight = class_weights,validation_data=[x_val,y_val],verbose=2)
    #auc_eval,
    
    print('training end...')
    
    return training

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.2, random_state=42)

In [60]:
ModelName = '/home/jujun/fraudprediction_10k/HAN/hanmodel_dropout_20200212'

In [65]:
y_train = pd.Series(y_train)

In [66]:
y_train.shape

(45030,)

In [67]:
y_test= pd.Series(y_test)

In [68]:
basicmodel = basicModel(embedding_matrix,MAX_NB_WORDS,MAX_PARA_LENGTH)
training = trainModel(X_train,y_train,ModelName, basicmodel, X_test, y_test)

weights (None, 250)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 250, 100)     5000100     input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 250, 100)     400         embedding_3[0][0]                
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 250, 100)     0           batch_normalization_9[0][0]      
________________________________________________________________________

 - 1148s - loss: 0.0104 - acc: 0.9988 - val_loss: 0.0545 - val_acc: 0.9923
 epoch:14 AUC: 0.8632
 epoch:14 Avg_Prec: 0.3753

Epoch 00015: Avg_Prec did not improve from 0.42352
Epoch 16/50
 - 1140s - loss: 0.0112 - acc: 0.9986 - val_loss: 0.0573 - val_acc: 0.9936
 epoch:15 AUC: 0.8471
 epoch:15 Avg_Prec: 0.4188

Epoch 00016: Avg_Prec did not improve from 0.42352
Epoch 17/50
 - 1152s - loss: 0.0104 - acc: 0.9988 - val_loss: 0.0551 - val_acc: 0.9932
 epoch:16 AUC: 0.8581
 epoch:16 Avg_Prec: 0.4502

Epoch 00017: Avg_Prec improved from 0.42352 to 0.45025, saving model to /home/jujun/fraudprediction_10k/HAN/hanmodel_dropout_20200212
Epoch 18/50
 - 1134s - loss: 0.0101 - acc: 0.9987 - val_loss: 0.0542 - val_acc: 0.9927
 epoch:17 AUC: 0.8559
 epoch:17 Avg_Prec: 0.4361

Epoch 00018: Avg_Prec did not improve from 0.45025
Epoch 19/50
 - 1148s - loss: 0.0098 - acc: 0.9990 - val_loss: 0.0506 - val_acc: 0.9932
 epoch:18 AUC: 0.8589
 epoch:18 Avg_Prec: 0.4831

Epoch 00019: Avg_Prec improved from 0.45

In [ ]:
basicmodel.load_weights(ModelName)
pred = basicmodel.predict(X_test)

In [ ]:
ap_test = average_precision_score(y_test, pred)
ap_test

In [ ]:
auc_test = roc_auc_score(y_test, pred)
auc_test